In [1]:
!git clone https://github.com/d909b/cxplain.git


Cloning into 'cxplain'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 114 (delta 18), reused 16 (delta 16), pack-reused 92 (from 1)
Receiving objects: 100% (114/114), 250.64 KiB | 8.95 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [1]:
%cd cxplain

/content/cxplain


In [5]:
!find . -type f -name "*.py" -exec sed -i 's/from tensorflow\.python\.keras\.layers/from tensorflow.keras.layers/g' {} +

In [2]:
!pip install wget

In [2]:
from cxplain import MLPModelBuilder, ZeroMasking, CXPlain

In [3]:
import os
import tensorflow as tf
from PIL import Image
import wget
import matplotlib.pyplot as plt
import numpy as np
import glob
import tarfile
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#wget.download('https://raw.githubusercontent.com/torch/tutorials/master/7_imagenet_classification/synset_words.txt')

filename = wget.download('https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-160.tgz')

if filename.endswith("tgz"):
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall()

extracted_path = './imagewoof2-160/train/'

target_size = (40, 40)

images = []
labels = []
label_dict = {}
label_index = 0

for class_folder in os.listdir(extracted_path):
    folder_path = os.path.join(extracted_path, class_folder)
    if os.path.isdir(folder_path):


        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            if image_file.endswith('.JPEG'):

                image = Image.open(image_path).convert('RGB')
                image = image.resize(target_size)
                image_array = np.array(image)

                if image_array.shape == (40, 40, 3):
                    images.append(image_array)
                    labels.append(class_folder)


images_np = np.array(images)
labels_np = np.array(labels)

In [5]:
input_shape = (40, 40, 3)

bb_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)


In [6]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [7]:
# Dividir los datos en nuevos conjuntos de entrenamiento y validación
x_train, x_test, y_train, y_test = train_test_split(images_np[0:500] , labels_np[0:500], test_size=0.30, random_state=42)


In [ ]:
# Reemplazar 'import collections' por 'from collections.abc import Sequence, Iterable'
!find . -type f -name "*.py" -exec sed -i 's/^import collections$/from collections.abc import Sequence, Iterable/' {} +

# Reemplazar 'collections.Sequence' por 'Sequence'
!find . -type f -name "*.py" -exec sed -i 's/collections\.Sequence/Sequence/g' {} +

# Reemplazar 'collections.Iterable' por 'Iterable'
!find . -type f -name "*.py" -exec sed -i 's/collections\.Iterable/Iterable/g' {} +

print("Reemplazo completado: 'import collections' por 'from collections.abc import Sequence, Iterable' y referencias actualizadas.")


In [8]:
from tensorflow.python.keras.losses import categorical_crossentropy
from cxplain import MLPModelBuilder, ZeroMasking, CXPlain
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout


explained_model = bb_model

# Define the model you want to use to explain your __explained_model__.
# Here, we use a neural explanation model with a
# multilayer perceptron (MLP) architecture.
model_builder = MLPModelBuilder(num_layers=2, num_units=64, batch_size=256, learning_rate=0.001)

# Define your masking operation - the method of simulating the
# removal of input features used internally by CXPlain - ZeroMasking is typically a sensible default choice for tabular and image data.
masking_operation = ZeroMasking()

# Define the loss with which each input features' associated reduction in prediction error is calculated.
loss = categorical_crossentropy

# Build and fit a CXPlain instance.
explainer = CXPlain(explained_model, model_builder, masking_operation, loss)


In [10]:
!grep -r "collections.Sequence" /content/cxplain/cxplain
!grep -r "collections.Iterable" /content/cxplain/cxplain


In [11]:
!pip uninstall -y cxplain


In [13]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from cxplain.backend.model_builders.base_model_builder import BaseModelBuilder

class MyModelBuilder(BaseModelBuilder):
    def build(self, input_layer):
        """
        Implementa la arquitectura específica de tu modelo aquí.
        """
        x = Conv2D(32, (3, 3), activation='relu', name='conv_layer')(input_layer)
        x = MaxPooling2D((2, 2), name='max_pool')(x)
        x = Flatten(name='flatten')(x)
        x = Dense(64, activation='relu', name='dense_layer')(x)
        return x  # Retorna la última capa antes de las capas de salida

    def _fit_single(self, model, X, y, masked_data=None):
        """
        Implementa cómo ajustar un solo modelo.
        """
        model.fit(
            X,
            [y, y, y],  # Ajustar según las salidas del modelo
            batch_size=self.batch_size,
            epochs=self.num_epochs,
            validation_split=self.validation_fraction,
            shuffle=self.shuffle,
            callbacks=self.callbacks
        )

    def _predict_single(self, model, X):
        """
        Implementa cómo predecir con un solo modelo.
        """
        return model.predict(X)

    def score(self, X, y, sample_weight=None):
        """
        Implementa cómo evaluar el modelo.
        """
        return self.explained_model.evaluate(X, y, sample_weight=sample_weight)


In [16]:
from cxplain import CXPlain

# Supongamos que ya tienes `explained_model`, `MyModelBuilder`, y `ZeroMasking` definidos
model_builder = MyModelBuilder()
masking_operation = ZeroMasking()

explainer = CXPlain(
    explained_model=explained_model,
    model_builder=model_builder,
    masking_operation=masking_operation,
    loss='categorical_crossentropy',
    downsample_factors=(1,),  # Ajusta según tus necesidades
    num_models=1  # Asegúrate de que este valor es 1 para un solo modelo
)

# Preprocesar tus datos según sea necesario
# Por ejemplo:
# x_train_preprocessed = preprocess_input(x_train)
# y_train_one_hot = to_categorical(y_train, num_classes=num_classes)

# Ajustar el explainer
explainer.fit(x_train, y_train)


input_layer: <KerasTensor shape=(None, 40, 40, 3), dtype=float32, sparse=None, name=keras_tensor_9802>
all_auxiliary_outputs: <KerasTensor shape=(None, 9), dtype=float32, sparse=None, name=all>
all_but_one_auxiliary_outputs_input: <KerasTensor shape=(None, 1600, 9), dtype=float32, sparse=None, name=all_but_one>


AttributeError: 'KerasHistory' object has no attribute 'layer'

In [9]:
explainer.fit(x_train, y_train)

# Use the __explainer__ to obtain explanations for the predictions of your __explained_model__.
attributions = explainer.explain(x_test)

input_layer: <KerasTensor shape=(None, 40, 40, 3), dtype=float32, sparse=None, name=input_layer>
all_auxiliary_outputs: <KerasTensor shape=(None, 9), dtype=float32, sparse=None, name=all_auxiliary_outputs>
all_but_one_auxiliary_outputs_input: <KerasTensor shape=(None, 1600, 9), dtype=float32, sparse=None, name=all_but_one_auxiliary_outputs_input>


AttributeError: 'KerasHistory' object has no attribute 'layer'

In [12]:
!find . -type f -name "*.py" -exec sed -i 's/from tensorflow\.python\.keras\.layers/from tensorflow.keras.layers/g' {} +
print("Reemplazo completado: 'from tensorflow.python.keras.layers' por 'from tensorflow.keras.layers'.")


Reemplazo completado: 'from tensorflow.python.keras.layers' por 'from tensorflow.keras.layers'.
